https://www.kdnuggets.com/introduction-ray-swiss-army-knife-distributed-computing

Today, applications handle large datasets and complex tasks. To meet these demands, many frameworks for distributed computing have been developed to speed up processes and reduce delays. One such popular framework is Ray. Ray is a flexible tool designed for cloud-based distributed computing and for building scalable machine learning systems. This article explores Ray, its key features, and its applications.

ONLY WORKS IN 'base (Python 3.11.7)' miniconda "kernel" on my computer.  Libraries aren't recognized in 'base (Python) 3.12.17' anaconda.

Ray is an open-source tool for distributed computing. It helps run tasks on multiple computers. Ray handles large datasets and complex operations. It makes scaling applications easier. Key features include an easy API, scalability, fault tolerance, and support for tasks like machine learning. Ray has libraries like Ray Core, Ray Data, Ray Train, Ray Tune, Ray Serve, and Ray RLlib. Each helps with specific tasks like data processing and model training.

Use Cases of Ray
 
Ray is a tool that can be used in many different scenarios.

1. Distributed Machine Learning: Ray speeds up machine learning model training across multiple computers. It is great for large datasets and complex models, especially in deep learning and reinforcement learning.
2. Hyperparameter Tuning: Ray Tune helps optimize machine learning models by testing different combinations of parameters. It speeds up the process of finding the best settings.
3. Model Serving: Ray Serve deploys machine learning models for real-time predictions. It scales dynamically to handle different loads with low latency.
4. Reinforcement Learning: Ray RLlib trains reinforcement learning models. It supports multiple algorithms and scales across machines for large, complex models.

In [1]:
import pandas as pd
import numpy as np


Ray Core
Ray Core is the base of the Ray framework. It helps build distributed applications. It also handles task scheduling and object management. Ray Core makes sure tasks run even if something fails. You can use it to run functions on many machines at once.

In [ ]:
import ray
# Initialize Ray
ray.init()

# Define a simple function to be parallelized
@ray.remote
def my_function(x):
    return x * x

# Run the function in parallel
futures = [my_function.remote(i) for i in range(10)]
results = ray.get(futures)

print(results)  # Output: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

ray.shutdown()

Ray Data
Ray Data provides abstractions to distribute data processing tasks, such as reading and preprocessing large datasets. It can scale tasks like data transformation, cleaning, and aggregation across multiple nodes.

Install Ray Data using the following command: pip install -U 'ray[data]'

Example: Scaling Data Processing with Ray Data

In [ ]:
from ray.data import read_csv

# Initialize Ray
ray.init()

# Load a large dataset
dataset = read_csv("large_dataset.csv")

# Apply transformation (filtering, mapping)
filtered = dataset.filter(lambda row: row["value"] > 10)
aggregated = filtered.groupby("category").sum("value")

# Show processed results
print(aggregated.take(10))

ray.shutdown()

Ray Train
Ray Train helps train machine learning models across many machines. It makes training faster by spreading the work over multiple nodes. This is useful for large datasets and complex models.

Install Ray Train using the following command: pip install -U "ray[train]"

Example: Scaling Machine Learning Training with Ray Train

In [ ]:
from ray.train import Trainer
from ray.train.sklearn import SklearnTrainer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Initialize Ray
ray.init()

# Load a sample dataset
X, y = load_iris(return_X_y=True)

# Define training function
def train_model():
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)
    return model

# Use SklearnTrainer to scale training
trainer = SklearnTrainer(train_func=train_model)
trainer.fit()

ray.shutdown()

Ray Tune
Ray Tune is a tool for hyperparameter tuning. It can test many combinations at the same time. You can use methods like grid search or random search. It also supports advanced methods like Bayesian optimization. Ray Tune helps optimize models quickly and efficiently.

Install Ray Tune using the following command: pip install "ray[tune]"

Example: Scaling Hyperparameter Tuning with Ray Tune

In [ ]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler

# Define training function with hyperparameters
def train_model(config):
    learning_rate = config["learning_rate"]
    for step in range(100):
        loss = (learning_rate * step) ** 0.5
        tune.report(loss=loss)

# Initialize Ray
ray.init()

# Run hyperparameter tuning with Ray Tune
analysis = tune.run(
    train_model,
    config={
        "learning_rate": tune.loguniform(1e-4, 1e-1),
    },
    scheduler=ASHAScheduler(metric="loss", mode="min"),
)

print("Best config: ", analysis.best_config)
ray.shutdown()

Ray Serve
Ray Serve is a tool for scaling model serving. It helps serve machine learning models in a distributed manner with dynamic scaling, load balancing, and low latency.

Install Ray Tune using the following command: pip install "ray[serve]"

Example: Scaling Model Serving with Ray Serve

In [ ]:
from ray import serve
import requests

# Initialize Ray Serve
serve.start()

# Define a model deployment
@serve.deployment
def model(request):
    return {"message": "Hello, Ray Serve!"}

# Deploy the model
model.deploy()

# Send a request to the model
response = requests.get("http://127.0.0.1:8000/model")
print(response.json())

ray.shutdown()

Ray RLlib
Ray RLlib helps train reinforcement learning models on multiple machines. It supports different algorithms, like Proximal Policy Optimization (PPO) and Deep Q-Network (DQN). These algorithms help teach models to make decisions based on rewards and actions.

Install Ray Tune using the following command: pip install "ray[rllib]"

Example: Scaling Reinforcement Learning with Ray RLlib

In [ ]:
from ray.rllib.algorithms.ppo import PPO

# Initialize Ray
ray.init()

# Define configuration for RL agent
config = {
    "env": "CartPole-v1",
    "framework": "torch",  # or "tf"
    "num_workers": 4,  # Number of parallel workers
}

# Train a PPO agent
trainer = PPO(config=config)
for _ in range(10):
    result = trainer.train()
    print(f"Episode reward: {result['episode_reward_mean']}")

ray.shutdown()

In [37]:
df = pd.read_csv(r'C:\PyProjects\OSBP Insights\data\raw\cleansed\data_source.csv')
# obsp_forecast_df = pd.read_csv(r'C:\PyProjects\OSBP Insights\references\forecast_listing\osbp_dashboard_forecast.csv')
# amc_forecast_df = pd.read_csv(r'C:\PyProjects\OSBP Insights\references\forecast_listing\amc_forecast_listing.csv')

In [38]:
# Filter acc_ri_awards_df to not inlcude any rows with a value of 'MODFICATION", "SATOC", AND "MATOC" in the 'Contract Action Type' column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 56 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Contract No                              29329 non-null  object 
 1   Order No                                 29329 non-null  object 
 2   Modification No                          29329 non-null  object 
 3   Award Date                               29329 non-null  object 
 4   Fiscal Year                              29329 non-null  int64  
 5   Command                                  29329 non-null  object 
 6   Sub Command                              29329 non-null  object 
 7   Organization                             29329 non-null  object 
 8   Office                                   29329 non-null  object 
 9   Office Id                                29329 non-null  object 
 10  Army Hierarchy                           29024